In [3]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     --------------------------------------- 0.0/15.3 MB 330.3 kB/s eta 0:00:47
     --------------------------------------- 0.0/15.3 MB 330.3 kB/s eta 0:00:47
     --------------------------------------- 0.0/15.3 MB 393.8 kB/s eta 0:00:39
     --------------------------------------- 0.1/15.3 MB 409.6 kB/s eta 0:00:38
     --------------------------------------- 0.1/15.3 MB 516.7 kB/s eta 0:00:30
      -------------------------------------- 0.2/15.3 MB 811.5 kB/s eta 0:00:19
      --------------------------------------- 0.4/15.3 MB 1.2 MB/s eta 0:00:13
     - -------------------------------------- 0.6/15.3 MB 1.6 MB/s eta 0:00:10
     -- ------------------------------------- 0.9/15.3 MB 2.2 MB/s eta 0:00:07
     --- ------------------------------------ 1.4/15.3 MB 3.0 MB/s eta 0:00:05
     --- ------------------------------------ 1.4/15.3 MB 3.0 MB/s eta 0:00:05
     ---- ----------------------------------- 1.6/15.

In [4]:
import spacy
import pandas as pd

from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.load("ru_core_news_sm")

In [5]:
df_train = pd.read_csv('train.csv', encoding="utf8")
df_val = pd.read_csv('valid.csv', encoding="utf8")

In [6]:
df_train = df_train.sample(frac=0.5)
df_val = df_val.sample(frac=0.5)

In [7]:
df_train

,Unnamed: 0,text,sentiment
73752,94850,Отличное расположение - всего 3 минуты пешком ...,1
173598,194696,"Очень люблю это место , считаю лучшим в городе...",1
94350,115448,"Совсем невкусно . Меню огромное , а толку нол...",2
38739,59837,росто при СССР татары расселялись везде вместе...,0
42838,63936,Отель постарались сделать необычным. Каждая Ко...,1
...,...,...,...
48648,69746,Ищите тщательней информацию про способы переме...,1
16041,37139,Агрессивный педиатр и плохой сервис Мы были на...,2
971,22069,"Все понравилось. Приедем еще!!! Единственное, ...",1
154025,175123,"Достижение тут - то, что уличный протест форми...",0


In [8]:
df_train['sentiment'].value_counts()

sentiment
1    45309
2    25036
0    24601
Name: count, dtype: int64

In [9]:
df_train = df_train[df_train['sentiment'] != 0]
df_val = df_val[df_val['sentiment'] != 0]

In [10]:
train_data = [tuple(df_train.iloc[i].values)[1:] for i in range(df_train.shape[0])]
valid_data = [tuple(df_val.iloc[i].values)[1:] for i in range(df_val.shape[0])]

In [11]:
train_data

[('Отличное расположение - всего 3 минуты пешком до голубой мечети. Просторные и светлые номера. Персонал очень отзывчивый - Бирол (с ресепшн) всегда на позитиве и подробно отвечает на любые вопросы. Отличное соотношение цены и качества',
  1),
 ('Очень люблю это место , считаю лучшим в городе , где делают суши .  Но вот очень опечалили вчера , когда заказали суши с собой .  Во первых забыли положить порцию мидий , и самое ужасное , не положили соевый соус ....  ',
  1),
 ('Совсем невкусно .  Меню огромное , а толку ноль … Лучше бы предлагали втрое меньше блюд , но умели их приготовить .  Суси и роллы еще ничего , но бывает и гораздо вкуснее , нежнее , свежее .  Хорошее обслуживание , хороший интерьер , а повара – на мыло . ( это всё про « Нэцкэ » Лениский пр-т , 69 )  ',
  2),
 ('Отель постарались сделать необычным. Каждая Комната здесь индивидуальная и имеет своё название. Санузлы полностью в мраморе. Комнаты насыщены дизайнерскими элементами. А матрас.. очень удобный. Здоровый шикар

In [12]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    # a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label==1:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        else:
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        # we need to set the (text)cat(egory) for each document
        #doc.cats["positive"] = label
        # put them into a nice list
        docs.append(doc)
    return docs

In [13]:
# we are so far only interested in the first 5000 reviews
# this will keep the training time short.
# In practice take as much data as you can get.
# you can always reduce it to make the script even faster.
num_texts = 5000
# first we need to transform all the training data
train_docs = make_docs(train_data[:num_texts])

  5%|▌         | 256/5000 [00:06<02:05, 37.91it/s]


KeyboardInterrupt: 

In [ ]:

# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

100%|██████████| 5000/5000 [01:31<00:00, 54.89it/s]


на этом месте мы идем в https://spacy.io/usage/training#quickstart, там настраиваем под себя конфиг (textcat), копируем его руками(!) в base_config.cfg, указываем правильные пути до трейн и вэлид


In [15]:
!python -m spacy train base_config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       39.01    0.39
  0     200         47.65       44.00    0.44
  0     400         41.33       74.82    0.75
  0     600         33.37       76.85    0.77
  0     800         30.82       77.78    0.78
  0    1000         32.65       78.24    0.78
  0    1200         31.77       81.01    0.81
  0    1400         28.43       82.28    0.82
  0    1600         27.62       82.46    0.82
  0    1800         25.85       83.39    0.83
  0    2000         24.85       84.05    0.84
  0    2200         23.12       82.88    0.83
  1    2400         19.96       84.45    0.84
  1    2600          6.37   

In [ ]:
import spacy
# load thebest model from training
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

type : ‘quit’ to exit
{'positive': 0.40170586109161377, 'negative': 0.5982941389083862}
the sentiment is negative
{'positive': 0.3088458180427551, 'negative': 0.6911541819572449}
the sentiment is negative
{'positive': 0.36228862404823303, 'negative': 0.6377114057540894}
the sentiment is negative
{'positive': 0.5347373485565186, 'negative': 0.46526259183883667}
the sentiment is positive
{'positive': 0.6125245690345764, 'negative': 0.3874754309654236}
the sentiment is positive
{'positive': 0.37885987758636475, 'negative': 0.6211401224136353}
the sentiment is negative
{'positive': 0.42735111713409424, 'negative': 0.5726488828659058}
the sentiment is negative
{'positive': 0.4019811749458313, 'negative': 0.5980188846588135}
the sentiment is negative
{'positive': 0.4652771055698395, 'negative': 0.5347228646278381}
the sentiment is negative
{'positive': 0.6100626587867737, 'negative': 0.3899373412132263}
the sentiment is positive
{'positive': 0.0, 'negative': 0.0}
the sentiment is negative
{'